In [3]:
import pandas as pd
import numpy as np

In [4]:
path="C:/Users/manas/my-project/cleaned_legal_metadata.csv"
df=pd.read_csv(path)
print(df.head())

         id                                               name  \
0  12532561  Jessica OLSEN, on behalf of herself and all ot...   
1  12532560  AMERICAN CENTER FOR LAW AND JUSTICE, Plaintiff...   
2  12532562  UNITED STATES of America, Plaintiff, v. Steven...   
3  12532563  Kaori STEARNEY, et al., Plaintiffs, v. UNITED ...   
4  12532564  IN RE: GERMAN AUTOMOTIVE MANUFACTURERS ANTITRU...   

                                   name_abbreviation decision_date  \
0                              Olsen v. Nelnet, Inc.    2019-05-21   
1  Am. Ctr. for Law & Justice v. U.S. Dep't of Ju...    2019-06-30   
2                             United States v. Emery    2019-06-21   
3                          Stearney v. United States    2019-05-16   
4          In re German Auto. Mfrs. Antitrust Litig.    2019-06-17   

                    docket_number court/name_abbreviation  
0                    4:18-CV-3081                 D. Neb.  
1  Civil Action No. 16-2188 (TJK)                  D.D.C.  
2 

In [5]:
print(df.tail())

          id                                               name  \
84  12532643  WORLD WATER WORKS HOLDINGS, INC., Plaintiff, v...   
85  12532644  Caitlin BERNARD M.D., Plaintiff, v. INDIVIDUAL...   
86  12532645  REXING QUALITY EGGS, Plaintiff, v. REMBRANDT E...   
87  12532646  Osha JOSEPH, Plaintiff, v. Sgt. Bobby DONAHUE,...   
88  12532647  ZURICH AMERICAN INSURANCE COMPANY, Plaintiff, ...   

                                    name_abbreviation decision_date  \
84  World Water Works Holdings, Inc. v. Cont'l Cas...    2019-06-24   
85  Bernard v. Individual Members of the Ind. Med....    2019-06-28   
86                    Eggs v. Rembrandt Enters., Inc.    2019-05-29   
87                                  Joseph v. Donahue    2019-05-28   
88          Zurich Am. Ins. Co. v. Ins. Co. of N. Am.    2019-05-21   

                docket_number court/name_abbreviation  
84             No. 17 CV 5237               N.D. Ill.  
85  No. 1:19-cv-01660-SEB-DML               S.D. Ind.  
86

In [6]:
print(df.columns)
print(df.isnull().sum())

Index(['id', 'name', 'name_abbreviation', 'decision_date', 'docket_number',
       'court/name_abbreviation'],
      dtype='object')
id                         0
name                       0
name_abbreviation          0
decision_date              0
docket_number              0
court/name_abbreviation    0
dtype: int64


In [9]:
print(df.shape)

(89, 6)


In [11]:
print(df.dtypes)

id                          int64
name                       object
name_abbreviation          object
decision_date              object
docket_number              object
court/name_abbreviation    object
dtype: object


In [17]:
Y

0       D. Neb.
1        D.D.C.
2       D. S.D.
3      D. Ariz.
4     N.D. Cal.
        ...    
84    N.D. Ill.
85    S.D. Ind.
86    S.D. Ind.
87     D. Minn.
88     E.D. Mo.
Name: court/name_abbreviation, Length: 89, dtype: object

In [18]:
X.describe()

,id
count,8.900000e+01
mean,1.253260e+07
std,2.583602e+01
min,1.253256e+07
25%,1.253258e+07
50%,1.253260e+07
75%,1.253262e+07
max,1.253265e+07


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load your CSV file
df = pd.read_csv("cleaned_legal_metadata.csv")

# Step 1: Derive 'case_type' from 'docket_number'
def extract_case_type(docket):
    docket = str(docket).upper()
    if 'CR' in docket:
        return 'Criminal'
    elif 'CV' in docket or 'CIVIL' in docket:
        return 'Civil'
    elif 'MDL' in docket:
        return 'MDL'
    else:
        return 'Other'

df['case_type'] = df['docket_number'].apply(extract_case_type)

# Step 2: Prepare text features and labels
X_text = df['name'].astype(str)
y = df['case_type']

# Step 3: Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(X_text)

# Step 4: Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 5: Train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Step 6: Predict and evaluate
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

       Civil       0.65      1.00      0.79        11
    Criminal       1.00      0.20      0.33         5
       Other       0.00      0.00      0.00         2

    accuracy                           0.67        18
   macro avg       0.55      0.40      0.37        18
weighted avg       0.67      0.67      0.57        18



c:\Users\manas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\manas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\manas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the CSV file
df = pd.read_csv("cleaned_legal_metadata.csv")

# Extract case type from docket number
def extract_case_type(docket):
    docket = str(docket).upper()
    if 'CR' in docket:
        return 'Criminal'
    elif 'CV' in docket or 'CIVIL' in docket:
        return 'Civil'
    elif 'MDL' in docket:
        return 'MDL'
    else:
        return 'Other'

df['case_type'] = df['docket_number'].apply(extract_case_type)

# Focus on major classes only
df = df[df['case_type'].isin(['Civil', 'Criminal', 'Other'])]

# Prepare features and labels
X_text = df['name'].astype(str)
y = df['case_type']

# Vectorize case name text
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(X_text)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred))


Random Forest Classification Report:
              precision    recall  f1-score   support

       Civil       0.75      1.00      0.86        12
    Criminal       1.00      0.25      0.40         4
       Other       1.00      0.50      0.67         2

    accuracy                           0.78        18
   macro avg       0.92      0.58      0.64        18
weighted avg       0.83      0.78      0.73        18

